In [59]:



class Morph:
    
    def __init__(self, surface, base, pos, pos1):
        '''初期化'''
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        '''オブジェクトの文字列表現'''
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(self.surface, self.base, self.pos, self.pos1)


def neco_lines():
    
    with open('neko.txt.cabocha') as fcabo:

        morphs = []
        for line in fcabo:

            # 1文の終了判定
            if line == 'EOS\n':
                yield morphs
                morphs = []

            else:
                # *はスキップ
                if line[0] == '*':
                    continue

                # 表層形はtab区切り、それ以外は','区切りでバラす
                cols = line.split('\t')
                res_cols = cols[1].split(',')

                # Morph作成、リストに追加
                morphs.append(Morph(
                    cols[0],        # surface
                    res_cols[6],    # base
                    res_cols[0],    # pos
                    res_cols[1]     # pos1
                ))

        raise StopIteration


# 1文ずつリスト作成
for i, morphs in enumerate(neco_lines(), 1):

    # 3文目を表示
    if i == 3:
        for morph in morphs:
            print(morph)
        break

surface[　]	base[　]	pos[記号]	pos1[空白]
surface[吾輩]	base[吾輩]	pos[名詞]	pos1[代名詞]
surface[は]	base[は]	pos[助詞]	pos1[係助詞]
surface[猫]	base[猫]	pos[名詞]	pos1[一般]
surface[で]	base[だ]	pos[助動詞]	pos1[*]
surface[ある]	base[ある]	pos[助動詞]	pos1[*]
surface[。]	base[。]	pos[記号]	pos1[句点]


In [5]:
#40に加えて，文節を表すクラスChunkを実装せよ．
#このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），
#係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．
#さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，
#8文目の文節の文字列と係り先を表示せよ．
#第5章の残りの問題では，ここで作ったプログラムを活用せよ．

import re

class Morph:
    
    def __init__(self, surface, base, pos, pos1):
        '''初期化'''
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        '''オブジェクトの文字列表現'''
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(self.surface, self.base, self.pos, self.pos1)

class Chunk:
    def __init__(self):
        self.morphs=[]
        self.srcs=[]
        self.dst=-1
        
    def __str__(self):
        surface=' '
        for morph in self.morphs:
            surface +=morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface,self.srcs,self.dst)
    
        
        

def neco_lines():
    '''「吾輩は猫である」の係り受け解析結果のジェネレータ
    「吾輩は猫である」の係り受け解析結果を順次読み込んで、
    1文ずつChunkクラスのリストを返す

    戻り値：
    1文のChunkクラスのリスト
    '''
    with open('neko.txt.cabocha') as fcabo:

        chunks = dict()     # idxをkeyにChunkを格納
        idx = -1

        for line in fcabo:

            # 1文の終了判定
            if line == 'EOS\n':

                # Chunkのリストを返す
                if len(chunks) > 0:

                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]
                    chunks.clear()

                else:
                    yield []

            # 先頭が*の行は係り受け解析結果なので、Chunkを作成
            elif line[0] == '*':

                # Chunkのインデックス番号と係り先のインデックス番号取得
                cols = line.split(' ')
                idx = int(cols[1])
                dst = int(re.search(r'(.*?)D', cols[2]).group(1))

                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst

                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)

            # それ以外の行は形態素解析結果なので、Morphを作りChunkに追加
            else:

                # 表層形はtab区切り、それ以外は','区切りでバラす
                cols = line.split('\t')
                res_cols = cols[1].split(',')

                # Morph作成、リストに追加
                chunks[idx].morphs.append(
                    Morph(
                        cols[0],        # surface
                        res_cols[6],    # base
                        res_cols[0],    # pos
                        res_cols[1]     # pos1
                    )
                )

        raise StopIteration


# 1文ずつリスト作成
for i, chunks in enumerate(neco_lines(), 1):

    # 8文目を表示
    if i == 8:
        for j, chunk in enumerate(chunks):
            print('[{}]{}'.format(j, chunk))
        break

[0] 吾輩は	srcs[]	dst[5]
[1] ここで	srcs[]	dst[2]
[2] 始めて	srcs[1]	dst[3]
[3] 人間という	srcs[2]	dst[4]
[4] ものを	srcs[3]	dst[5]
[5] 見た。	srcs[0, 4]	dst[-1]


In [3]:
#係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．
#ただし，句読点などの記号は出力しないようにせよ

In [4]:
#名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．
#ただし，句読点などの記号は出力しないようにせよ．



In [5]:
#与えられた文の係り受け木を有向グラフとして可視化せよ．
#可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，
#Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

